In [ ]:
!pip install bertviz

In [ ]:
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

model_ckpt = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
text = 'time flies like an arrow'
show(model, 'bert', tokenizer, text, display_mode = 'light', layer = 0, head = 0)

In [ ]:
from transformers import AutoTokenizer

model_ckpt = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
text = 'time flies like an arrow'

In [ ]:
input = tokenizer(text, return_tensors = 'pt', add_special_tokens = False)
input.input_ids

In [ ]:
from torch import nn
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_ckpt)
token_embd = nn.Embedding(config.vocab_size, config.hidden_size)
token_embd

In [ ]:
inputs_embeds =  token_embd(input.input_ids)
inputs_embeds.size()

In [ ]:
import torch
from math import sqrt

query = key = value = inputs_embeds
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
scores.size()

In [ ]:
from torch.nn.functional import softmax

weights = softmax(scores, dim = -1)
weights.sum(dim = -1)

In [ ]:
weights

In [ ]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

In [ ]:
def scaled_dot_product_attention(query, key, value):
  dim_k = key.size(-1)
  scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
  weights = softmax(scores, dim = -1)
  return torch.bmm(weights, value)

In [ ]:
class AttentionHead(nn.Module):
  def __init__(self, embed_dim, head_dim):
    super().__init__()
    self.q = nn.Linear(embed_dim, head_dim)
    self.k = nn.Linear(embed_dim, head_dim)
    self.v = nn.Linear(embed_dim, head_dim)

  def forward(self, hidden_state):
    attn_outputs = scaled_dot_product_attention(
        self.q(hidden_state), self.k(hidden_state), self.v(hidden_state)
    )
    return attn_outputs

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, config):
    super().__init__()
    embed_dim = config.hidden_size
    num_heads = config.num_attention_heads
    head_dim = embed_dim // num_heads
    self.heads = nn.ModuleList(
        [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
    )
    self.output_linear = nn.Linear(embed_dim, embed_dim)

  def forward(self, hidden_state):
    x = torch.cat([h(hidden_state) for h in self.heads], dim = -1)
    x = self.output_linear(x)
    return x

In [ ]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embeds)
attn_output.size()

In [ ]:
from bertviz import head_view
from transformers import AutoModel

model = AutoModel.from_pretrained(model_ckpt, output_attentions = True)

sentence_a = 'time flies like an arrow'
sentence_b = 'fruit flies like a banana'

viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors = 'pt')
attention = model(**viz_inputs).attentions
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim = 1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

head_view(attention, tokens, sentence_b_start, heads = [8])

# The Feed-Forward Layer

In [ ]:
class FeedForward(nn.Module):
  def __init__(self, config):
    super(FeedForward, self).__init__()
    self.config = config
    self.linear1 = nn.Linear(config.hidden_size, config.intermediate_size)
    self.linear2 = nn.Linear(config.intermediate_size, config.hidden_size)
    self.gelu = nn.GELU()
    self.dropout = nn.Dropout(config.hidden_dropout_prob)

  def forward(self, x):
    x = self.linear1(x)
    x = self.gelu(x)
    x = self.linear2(x)
    x = self.dropout(x)
    return x

In [ ]:
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_output)
ff_outputs.size()

In [ ]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.layer_norm1 = nn.LayerNorm(config.hidden_size)
    self.layer_norm2 = nn.LayerNorm(config.hidden_size)
    self.attention = MultiHeadAttention(config)
    self.feed_forward = FeedForward(config)

  def forward(self, x):
    hidden_state = self.layer_norm1(x)
    x = x + self.attention(hidden_state)
    x = x + self.feed_forward(self.layer_norm2(x))
    return x

In [ ]:
encoder_layer = TransformerEncoderLayer(config)
encoder_layer(inputs_embeds).size()

In [ ]:
class Embeddings(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size,config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings,config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size,eps = 1e-12)
        self.dropout = nn.Dropout()
    def forward(self,input_ids):
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length,dtype = torch.long).unsqueeze(0)
        token_embeddings = self.token_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        embeddings = token_embeddings+position_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [ ]:
embedding_layer = Embeddings(config)
embedding_layer(input.input_ids).size()

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.embeddings = Embeddings(config)
        self.layers = nn.ModuleList([TransformerEncoderLayer(config)]
                                    for _ in range(config.num_hidden_layers))

    def forward(self,x):
        x = self.embeddings(x)
        for layer in self.layers:
            x = layer(x)
        return x

# Adding a classification head

In [ ]:
class TransformerForSequenceClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = TransformerEncoder(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size,config.num_labels)

    def forward(self,x):
        x = self.encoder(x)[:,0,:]
        x = self.dropout(x)
        x = self.classifier(x)
        return x

In [ ]:
config.num_labels=3
encoder_classifier=TransformerForSequenceClassification(config)
encoder_classifier(input.input_ids).size()

# The Decoder

In [ ]:
seq_len=input.input_ids.size(-1)
mask=torch.tril(torch.ones(seq_len,seq_len)).unsqueeze(0)
mask[0]

In [ ]:
scores.masked_fill(mask==0,-float('inf'))

In [ ]:
def scaled_dot_product_attention(query,key,value,mask=None):
    dim_k=key.size(-1)
    scores=torch.bmm(query,key.transpose(1,2)/sqrt(dim_k))
    if mask is not None:
        scores=scores.masked_fill(mask,float('-inf'))
        weights=softmax(scores,dim=-1)
        return torch.bmm(weights,value)